# Cardiac Valve 4D Time-Series Conversion to USD

This notebook demonstrates converting time-varying cardiac valve simulation data from VTK format to animated USD.

## Dataset: CHOP-Valve4D

Two cardiac valve models with time-varying geometry:

- **Alterra**: 232 time steps (cardiac cycle simulation)
- **TPV25**: 265 time steps (cardiac cycle simulation)

These datasets represent 4D (3D + time) simulations of prosthetic heart valves during a cardiac cycle.

## Goals

1. Load and inspect time-varying VTK data
2. Convert entire time series to animated USD
3. Handle large datasets efficiently
4. Preserve all simulation data as USD primvars
5. Create multiple variations (full resolution, subsampled, etc.)

In [1]:
import sys
from pathlib import Path
import re
import time as time_module

# Add src to path
project_root = Path.cwd().parent.parent
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print(f"Project root: {project_root}")
print(f"Source path: {src_path}")

Project root: c:\src\Projects\PhysioMotion\physiomotion4d
Source path: c:\src\Projects\PhysioMotion\physiomotion4d\src


## Configuration

Control which time series conversions to compute.

In [2]:
# Configuration: Control which conversions to run
# Set to True to compute full time series (all frames) - takes longer
# Set to False to only compute subsampled time series (faster, for preview)
COMPUTE_FULL_TIME_SERIES = False  # Default: only subsampled

print("Time Series Configuration:")
print(f"  - Compute Full Time Series: {COMPUTE_FULL_TIME_SERIES}")
print("  - Compute Subsampled Time Series: Always enabled")
print()
if not COMPUTE_FULL_TIME_SERIES:
    print("⚠️  Full time series conversion is DISABLED for faster execution.")
    print("   Set COMPUTE_FULL_TIME_SERIES = True to enable full conversion.")
else:
    print("✓ Full time series conversion is ENABLED (this will take longer).")

Time Series Configuration:
  - Compute Full Time Series: False
  - Compute Subsampled Time Series: Always enabled

⚠️  Full time series conversion is DISABLED for faster execution.
   Set COMPUTE_FULL_TIME_SERIES = True to enable full conversion.


In [3]:
import logging
import numpy as np
from pxr import Usd, UsdGeom

# Import the vtk_to_usd library
from physiomotion4d.vtk_to_usd import (
    VTKToUSDConverter,
    ConversionSettings,
    MaterialData,
    read_vtk_file,
    validate_time_series_topology,
)

# Import USDTools for post-processing colormap
from physiomotion4d.usd_tools import USDTools

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

## 1. Discover and Organize Time-Series Files

In [4]:
# Define data directories
data_dir = project_root / "data" / "CHOP-Valve4D"
alterra_dir = data_dir / "Alterra"
tpv25_dir = data_dir / "TPV25"
output_dir = Path.cwd() / "output" / "valve4d"
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Data directory: {data_dir}")
print(f"Output directory: {output_dir}")
print("\nDirectory status:")
print(f"  Alterra: {'✓' if alterra_dir.exists() else '✗'} {alterra_dir}")
print(f"  TPV25: {'✓' if tpv25_dir.exists() else '✗'} {tpv25_dir}")

Data directory: c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D
Output directory: c:\src\Projects\PhysioMotion\physiomotion4d\experiments\convert_vtk_to_usd_lib\output\valve4d

Directory status:
  Alterra: ✓ c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\Alterra
  TPV25: ✓ c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\TPV25


In [5]:
def discover_time_series(directory, pattern=r"\.t(\d+)\.vtk$"):
    """Discover and sort time-series VTK files.

    Args:
        directory: Directory containing VTK files
        pattern: Regex pattern to extract time step number

    Returns:
        list: Sorted list of (time_step, file_path) tuples
    """
    vtk_files = list(Path(directory).glob("*.vtk"))

    # Extract time step numbers and pair with files
    time_series = []
    for vtk_file in vtk_files:
        match = re.search(pattern, vtk_file.name)
        if match:
            time_step = int(match.group(1))
            time_series.append((time_step, vtk_file))

    # Sort by time step
    time_series.sort(key=lambda x: x[0])

    return time_series


# Discover both datasets
alterra_series = discover_time_series(alterra_dir)
tpv25_series = discover_time_series(tpv25_dir)

print("=" * 60)
print("Time-Series Discovery")
print("=" * 60)
print("\nAlterra:")
print(f"  Files found: {len(alterra_series)}")
if alterra_series:
    print(f"  Time range: t{alterra_series[0][0]} to t{alterra_series[-1][0]}")
    print(f"  First file: {alterra_series[0][1].name}")
    print(f"  Last file: {alterra_series[-1][1].name}")

print("\nTPV25:")
print(f"  Files found: {len(tpv25_series)}")
if tpv25_series:
    print(f"  Time range: t{tpv25_series[0][0]} to t{tpv25_series[-1][0]}")
    print(f"  First file: {tpv25_series[0][1].name}")
    print(f"  Last file: {tpv25_series[-1][1].name}")

Time-Series Discovery

Alterra:
  Files found: 232
  Time range: t0 to t231
  First file: Alterra_output.t000.vtk
  Last file: Alterra_output.t231.vtk

TPV25:
  Files found: 265
  Time range: t0 to t264
  First file: TPV25_output.t000.vtk
  Last file: TPV25_output.t264.vtk


## 2. Inspect First Frame

Examine the first time step to understand the data structure.

In [6]:
# Read first frame of Alterra
if alterra_series:
    print("=" * 60)
    print("Alterra - First Frame Analysis")
    print("=" * 60)

    first_file = alterra_series[0][1]
    mesh_data = read_vtk_file(first_file, extract_surface=True)

    print(f"\nFile: {first_file.name}")
    print("\nGeometry:")
    print(f"  Points: {len(mesh_data.points):,}")
    print(f"  Faces: {len(mesh_data.face_vertex_counts):,}")
    print(f"  Normals: {'Yes' if mesh_data.normals is not None else 'No'}")
    print(f"  Colors: {'Yes' if mesh_data.colors is not None else 'No'}")

    # Bounding box
    bbox_min = np.min(mesh_data.points, axis=0)
    bbox_max = np.max(mesh_data.points, axis=0)
    bbox_size = bbox_max - bbox_min
    print("\nBounding Box:")
    print(f"  Min: [{bbox_min[0]:.3f}, {bbox_min[1]:.3f}, {bbox_min[2]:.3f}]")
    print(f"  Max: [{bbox_max[0]:.3f}, {bbox_max[1]:.3f}, {bbox_max[2]:.3f}]")
    print(f"  Size: [{bbox_size[0]:.3f}, {bbox_size[1]:.3f}, {bbox_size[2]:.3f}]")

    print(f"\nData Arrays ({len(mesh_data.generic_arrays)}):")
    for i, array in enumerate(mesh_data.generic_arrays, 1):
        print(f"  {i}. {array.name}:")
        print(f"     - Type: {array.data_type.value}")
        print(f"     - Components: {array.num_components}")
        print(f"     - Interpolation: {array.interpolation}")
        print(f"     - Elements: {len(array.data):,}")
        if array.data.size > 0:
            print(f"     - Range: [{np.min(array.data):.6f}, {np.max(array.data):.6f}]")

INFO: Reading legacy VTK file: c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\Alterra\Alterra_output.t000.vtk


Alterra - First Frame Analysis


INFO: Loaded mesh: 48482 points, 83634 faces, 4 data arrays



File: Alterra_output.t000.vtk

Geometry:
  Points: 48,482
  Faces: 83,634
  Normals: Yes
  Colors: No

Bounding Box:
  Min: [-28.819, -29.731, -72.836]
  Max: [39.019, 51.354, 65.425]
  Size: [67.837, 81.085, 138.261]

Data Arrays (4):
  1. displacement:
     - Type: float
     - Components: 3
     - Interpolation: vertex
     - Elements: 48,482
     - Range: [0.000000, 0.000000]
  2. shell_thickness:
     - Type: float
     - Components: 1
     - Interpolation: vertex
     - Elements: 48,482
     - Range: [0.000000, 2.000000]
  3. stress:
     - Type: float
     - Components: 9
     - Interpolation: vertex
     - Elements: 48,482
     - Range: [0.000000, 0.000000]
  4. relative_volume:
     - Type: float
     - Components: 1
     - Interpolation: vertex
     - Elements: 48,482
     - Range: [1.000000, 1.000000]


In [7]:
# Read first frame of TPV25
if tpv25_series:
    print("=" * 60)
    print("TPV25 - First Frame Analysis")
    print("=" * 60)

    first_file = tpv25_series[0][1]
    mesh_data = read_vtk_file(first_file, extract_surface=True)

    print(f"\nFile: {first_file.name}")
    print("\nGeometry:")
    print(f"  Points: {len(mesh_data.points):,}")
    print(f"  Faces: {len(mesh_data.face_vertex_counts):,}")
    print(f"  Normals: {'Yes' if mesh_data.normals is not None else 'No'}")
    print(f"  Colors: {'Yes' if mesh_data.colors is not None else 'No'}")

    # Bounding box
    bbox_min = np.min(mesh_data.points, axis=0)
    bbox_max = np.max(mesh_data.points, axis=0)
    bbox_size = bbox_max - bbox_min
    print("\nBounding Box:")
    print(f"  Min: [{bbox_min[0]:.3f}, {bbox_min[1]:.3f}, {bbox_min[2]:.3f}]")
    print(f"  Max: [{bbox_max[0]:.3f}, {bbox_max[1]:.3f}, {bbox_max[2]:.3f}]")
    print(f"  Size: [{bbox_size[0]:.3f}, {bbox_size[1]:.3f}, {bbox_size[2]:.3f}]")

    print(f"\nData Arrays ({len(mesh_data.generic_arrays)}):")
    for i, array in enumerate(mesh_data.generic_arrays, 1):
        print(f"  {i}. {array.name}:")
        print(f"     - Type: {array.data_type.value}")
        print(f"     - Components: {array.num_components}")
        print(f"     - Interpolation: {array.interpolation}")
        print(f"     - Elements: {len(array.data):,}")
        if array.data.size > 0:
            print(f"     - Range: [{np.min(array.data):.6f}, {np.max(array.data):.6f}]")

INFO: Reading legacy VTK file: c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\TPV25\TPV25_output.t000.vtk


TPV25 - First Frame Analysis


INFO: Loaded mesh: 38301 points, 63332 faces, 4 data arrays



File: TPV25_output.t000.vtk

Geometry:
  Points: 38,301
  Faces: 63,332
  Normals: Yes
  Colors: No

Bounding Box:
  Min: [-30.500, -31.550, -75.453]
  Max: [38.519, 48.022, 57.744]
  Size: [69.018, 79.573, 133.198]

Data Arrays (4):
  1. displacement:
     - Type: float
     - Components: 3
     - Interpolation: vertex
     - Elements: 38,301
     - Range: [0.000000, 0.000000]
  2. shell_thickness:
     - Type: float
     - Components: 1
     - Interpolation: vertex
     - Elements: 38,301
     - Range: [0.000000, 2.000000]
  3. stress:
     - Type: float
     - Components: 9
     - Interpolation: vertex
     - Elements: 38,301
     - Range: [0.000000, 0.000000]
  4. relative_volume:
     - Type: float
     - Components: 1
     - Interpolation: vertex
     - Elements: 38,301
     - Range: [1.000000, 1.000000]


In [8]:
# Note: Helper functions removed - now using USDTools for primvar inspection and colorization
# The workflow has changed to: convert to USD first, then apply colormap post-processing

# Configuration: choose colormap for visualization
DEFAULT_COLORMAP = "plasma"  # matplotlib colormap name

# Enable automatic colorization (will pick strain/stress primvars if available)
ENABLE_AUTO_COLORIZATION = True

print("Colorization will be applied after USD conversion using USDTools methods")
print("  - USDTools.list_mesh_primvars() for inspection")
print("  - USDTools.pick_color_primvar() for selection")
print("  - USDTools.apply_colormap_from_primvar() for coloring")
print(f"  - Colormap: {DEFAULT_COLORMAP}")

Colorization will be applied after USD conversion using USDTools methods
  - USDTools.list_mesh_primvars() for inspection
  - USDTools.pick_color_primvar() for selection
  - USDTools.apply_colormap_from_primvar() for coloring
  - Colormap: plasma


In [9]:
## 2. Configure Conversion Settings

# Create converter settings
settings = ConversionSettings(
    triangulate_meshes=True,
    compute_normals=False,  # Use existing normals if available
    preserve_point_arrays=True,
    preserve_cell_arrays=True,
    up_axis="Y",
    times_per_second=60.0,  # 60 FPS for smooth animation
    use_time_samples=True,
)

print("Conversion settings configured")
print(f"  - Triangulate: {settings.triangulate_meshes}")
print(f"  - FPS: {settings.times_per_second}")
print(f"  - Up axis: {settings.up_axis}")

Conversion settings configured
  - Triangulate: True
  - FPS: 60.0
  - Up axis: Y


## 3. Convert Full Time Series - Alterra

Convert the complete Alterra dataset to animated USD.

In [10]:
# Create material for Alterra
# Note: Vertex colors will be applied post-conversion by USDTools
alterra_material = MaterialData(
    name="alterra_valve",
    diffuse_color=(0.4, 0.5, 0.8),
    roughness=0.3,
    metallic=0.1,
    use_vertex_colors=False,  # USDTools will bind vertex color material during colorization
)

print("=" * 60)
print("Converting Alterra Time Series")
print("=" * 60)

Converting Alterra Time Series


In [11]:
# Convert Alterra (full resolution)
if COMPUTE_FULL_TIME_SERIES and alterra_series:
    converter = VTKToUSDConverter(settings)

    # Extract file paths and time codes
    alterra_files = [file_path for _, file_path in alterra_series]
    alterra_times = [float(time_step) for time_step, _ in alterra_series]

    output_usd = output_dir / "alterra_full.usd"

    print(f"\nConverting to: {output_usd}")
    print(f"Time codes: {alterra_times[0]:.1f} to {alterra_times[-1]:.1f}")
    print("\nThis may take several minutes...\n")

    start_time = time_module.time()

    # Read MeshData
    mesh_data_sequence = [read_vtk_file(f, extract_surface=True) for f in alterra_files]

    # Validate topology consistency across time series
    validation_report = validate_time_series_topology(
        mesh_data_sequence, filenames=alterra_files
    )
    if not validation_report["is_consistent"]:
        print(
            f"Warning: Found {len(validation_report['warnings'])} topology/primvar issues"
        )
        if validation_report["topology_changes"]:
            print(
                f"  Topology changes in {len(validation_report['topology_changes'])} frames"
            )

    # Convert to USD (preserves all primvars from VTK)
    stage = converter.convert_mesh_data_sequence(
        mesh_data_sequence=mesh_data_sequence,
        output_usd=output_usd,
        mesh_name="AlterraValve",
        time_codes=alterra_times,
        material=alterra_material,
    )

    # Repair elementSize for multi-component primvars (e.g. 9-component stress tensor)
    usd_tools = USDTools()
    mesh_path = "/World/Meshes/AlterraValve"
    repair_report = usd_tools.repair_mesh_primvar_element_sizes(
        str(output_usd), mesh_path
    )
    if repair_report["updated"]:
        print(f"Repaired elementSize for {len(repair_report['updated'])} primvar(s)")

    # Post-process: apply colormap visualization using USDTools
    if ENABLE_AUTO_COLORIZATION:
        # Inspect and select primvar for coloring
        primvars = usd_tools.list_mesh_primvars(str(output_usd), mesh_path)
        color_primvar = usd_tools.pick_color_primvar(
            primvars, keywords=("strain", "stress")
        )

        if color_primvar:
            print(f"\nApplying colormap to '{color_primvar}' using {DEFAULT_COLORMAP}")
            usd_tools.apply_colormap_from_primvar(
                str(output_usd),
                mesh_path,
                color_primvar,
                cmap=DEFAULT_COLORMAP,
                bind_vertex_color_material=True,
            )
        else:
            print("\nNo strain/stress primvar found for coloring")

    elapsed = time_module.time() - start_time

    print(f"\n✓ Conversion completed in {elapsed:.1f} seconds")
    print(f"  Output: {output_usd}")
    print(f"  Size: {output_usd.stat().st_size / (1024 * 1024):.2f} MB")
    print(f"  Time range: {stage.GetStartTimeCode()} - {stage.GetEndTimeCode()}")
    print(
        f"  Duration: {(stage.GetEndTimeCode() - stage.GetStartTimeCode()) / settings.times_per_second:.2f} seconds @ {settings.times_per_second} FPS"
    )
elif not COMPUTE_FULL_TIME_SERIES:
    print("⏭️  Skipping Alterra full time series (COMPUTE_FULL_TIME_SERIES = False)")

⏭️  Skipping Alterra full time series (COMPUTE_FULL_TIME_SERIES = False)


## 4. Convert Subsampled Time Series - Alterra

For faster previews, create a subsampled version (every Nth frame).

In [12]:
# Subsample Alterra (every 5th frame)
if alterra_series:
    subsample_rate = 5
    alterra_subsampled = alterra_series[::subsample_rate]

    print("=" * 60)
    print(f"Converting Subsampled Alterra (every {subsample_rate}th frame)")
    print("=" * 60)
    print(f"Frames: {len(alterra_series)} → {len(alterra_subsampled)}")

    converter = VTKToUSDConverter(settings)

    alterra_files_sub = [file_path for _, file_path in alterra_subsampled]
    alterra_times_sub = [float(time_step) for time_step, _ in alterra_subsampled]

    output_usd_sub = output_dir / f"alterra_subsample_{subsample_rate}x.usd"

    print(f"\nConverting to: {output_usd_sub}")

    start_time = time_module.time()

    # Read MeshData
    mesh_data_sequence = [
        read_vtk_file(f, extract_surface=True) for f in alterra_files_sub
    ]

    # Validate topology consistency across time series
    validation_report = validate_time_series_topology(
        mesh_data_sequence, filenames=alterra_files_sub
    )
    if not validation_report["is_consistent"]:
        print(
            f"Warning: Found {len(validation_report['warnings'])} topology/primvar issues"
        )
        if validation_report["topology_changes"]:
            print(
                f"  Topology changes in {len(validation_report['topology_changes'])} frames"
            )

    # Convert to USD (preserves all primvars from VTK)
    stage_sub = converter.convert_mesh_data_sequence(
        mesh_data_sequence=mesh_data_sequence,
        output_usd=output_usd_sub,
        mesh_name="AlterraValve",
        time_codes=alterra_times_sub,
        material=alterra_material,
    )

    # Repair elementSize for multi-component primvars (e.g. 9-component stress tensor)
    usd_tools = USDTools()
    mesh_path = "/World/Meshes/AlterraValve"
    repair_report = usd_tools.repair_mesh_primvar_element_sizes(
        str(output_usd_sub), mesh_path
    )
    if repair_report["updated"]:
        print(f"Repaired elementSize for {len(repair_report['updated'])} primvar(s)")

    # Post-process: apply colormap visualization using USDTools
    if ENABLE_AUTO_COLORIZATION:
        # Inspect and select primvar for coloring
        primvars = usd_tools.list_mesh_primvars(str(output_usd_sub), mesh_path)
        color_primvar = usd_tools.pick_color_primvar(
            primvars, keywords=("strain", "stress")
        )

        if color_primvar:
            print(f"\nApplying colormap to '{color_primvar}' using {DEFAULT_COLORMAP}")
            usd_tools.apply_colormap_from_primvar(
                str(output_usd_sub),
                mesh_path,
                color_primvar,
                cmap=DEFAULT_COLORMAP,
                bind_vertex_color_material=True,
            )
        else:
            print("\nNo strain/stress primvar found for coloring")

    elapsed = time_module.time() - start_time

    print(f"\n✓ Conversion completed in {elapsed:.1f} seconds")
    print(f"  Output: {output_usd_sub}")
    print(f"  Size: {output_usd_sub.stat().st_size / (1024 * 1024):.2f} MB")

INFO: Reading legacy VTK file: c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\Alterra\Alterra_output.t000.vtk


Converting Subsampled Alterra (every 5th frame)
Frames: 232 → 47

Converting to: c:\src\Projects\PhysioMotion\physiomotion4d\experiments\convert_vtk_to_usd_lib\output\valve4d\alterra_subsample_5x.usd


INFO: Loaded mesh: 48482 points, 83634 faces, 4 data arrays
INFO: Reading legacy VTK file: c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\Alterra\Alterra_output.t005.vtk
INFO: Loaded mesh: 48482 points, 83634 faces, 4 data arrays
INFO: Reading legacy VTK file: c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\Alterra\Alterra_output.t010.vtk
INFO: Loaded mesh: 48482 points, 83634 faces, 4 data arrays
INFO: Reading legacy VTK file: c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\Alterra\Alterra_output.t015.vtk
INFO: Loaded mesh: 48482 points, 83634 faces, 4 data arrays
INFO: Reading legacy VTK file: c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\Alterra\Alterra_output.t020.vtk
INFO: Loaded mesh: 48482 points, 83634 faces, 4 data arrays
INFO: Reading legacy VTK file: c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\Alterra\Alterra_output.t025.vtk
INFO: Loaded mesh: 48482 points, 83634 faces, 4 data arrays
INFO: Reading l


Applying colormap to 'vtk_point_stress_c0' using plasma


2026-01-29 21:22:28 INFO USDTools Wrote displayColor primvar with 47 time samples
2026-01-29 21:22:28 INFO USDTools Created vertex color material: /World/Looks/VertexColorMaterial
2026-01-29 21:22:28 INFO USDTools Saved USD file: c:\src\Projects\PhysioMotion\physiomotion4d\experiments\convert_vtk_to_usd_lib\output\valve4d\alterra_subsample_5x.usd



✓ Conversion completed in 64.8 seconds
  Output: c:\src\Projects\PhysioMotion\physiomotion4d\experiments\convert_vtk_to_usd_lib\output\valve4d\alterra_subsample_5x.usd
  Size: 198.03 MB


## 5. Convert Full Time Series - TPV25

In [13]:
# Create material for TPV25
# Note: Vertex colors will be applied post-conversion by USDTools
# Create material for TPV25
# Note: Vertex colors will be applied post-conversion by USDTools
tpv25_material = MaterialData(
    name="tpv25_valve",
    diffuse_color=(0.85, 0.4, 0.4),
    roughness=0.4,
    metallic=0.0,
    use_vertex_colors=False,  # USDTools will bind vertex color material during colorization
)

print("=" * 60)
print("Converting TPV25 Time Series")
print("=" * 60)
print(f"Dataset: {len(tpv25_series)} frames")

# Convert TPV25 (full resolution)
if COMPUTE_FULL_TIME_SERIES and tpv25_series:
    converter = VTKToUSDConverter(settings)

    tpv25_files = [file_path for _, file_path in tpv25_series]
    tpv25_times = [float(time_step) for time_step, _ in tpv25_series]

    output_usd = output_dir / "tpv25_full.usd"

    print(f"\nConverting to: {output_usd}")
    print(f"Time codes: {tpv25_times[0]:.1f} to {tpv25_times[-1]:.1f}")
    print("\nThis may take several minutes...\n")

    start_time = time_module.time()

    # Read MeshData
    mesh_data_sequence = [read_vtk_file(f, extract_surface=True) for f in tpv25_files]

    # Validate topology consistency across time series
    validation_report = validate_time_series_topology(
        mesh_data_sequence, filenames=tpv25_files
    )
    if not validation_report["is_consistent"]:
        print(
            f"Warning: Found {len(validation_report['warnings'])} topology/primvar issues"
        )
        if validation_report["topology_changes"]:
            print(
                f"  Topology changes in {len(validation_report['topology_changes'])} frames"
            )

    # Convert to USD (preserves all primvars from VTK)
    stage = converter.convert_mesh_data_sequence(
        mesh_data_sequence=mesh_data_sequence,
        output_usd=output_usd,
        mesh_name="TPV25Valve",
        time_codes=tpv25_times,
        material=tpv25_material,
    )

    # Post-process: apply colormap visualization using USDTools
    if ENABLE_AUTO_COLORIZATION:
        usd_tools = USDTools()
        mesh_path = "/World/Meshes/TPV25Valve"

        # Inspect and select primvar for coloring
        primvars = usd_tools.list_mesh_primvars(str(output_usd), mesh_path)
        color_primvar = usd_tools.pick_color_primvar(
            primvars, keywords=("strain", "stress")
        )

        if color_primvar:
            print(f"\nApplying colormap to '{color_primvar}' using {DEFAULT_COLORMAP}")
            usd_tools.apply_colormap_from_primvar(
                str(output_usd),
                mesh_path,
                color_primvar,
                cmap=DEFAULT_COLORMAP,
                bind_vertex_color_material=True,
            )
        else:
            print("\nNo strain/stress primvar found for coloring")

    elapsed = time_module.time() - start_time

    print(f"\n✓ Conversion completed in {elapsed:.1f} seconds")
    print(f"  Output: {output_usd}")
    print(f"  Size: {output_usd.stat().st_size / (1024 * 1024):.2f} MB")
    print(f"  Time range: {stage.GetStartTimeCode()} - {stage.GetEndTimeCode()}")
    print(
        f"  Duration: {(stage.GetEndTimeCode() - stage.GetStartTimeCode()) / settings.times_per_second:.2f} seconds @ {settings.times_per_second} FPS"
    )
elif not COMPUTE_FULL_TIME_SERIES:
    print("⏭️  Skipping TPV25 full time series (COMPUTE_FULL_TIME_SERIES = False)")

Converting TPV25 Time Series
Dataset: 265 frames
⏭️  Skipping TPV25 full time series (COMPUTE_FULL_TIME_SERIES = False)


## 6. Convert Subsampled Time Series - TPV25

In [14]:
# Subsample TPV25 (every 5th frame)
if tpv25_series:
    subsample_rate = 5
    tpv25_subsampled = tpv25_series[::subsample_rate]

    print("=" * 60)
    print(f"Converting Subsampled TPV25 (every {subsample_rate}th frame)")
    print("=" * 60)
    print(f"Frames: {len(tpv25_series)} → {len(tpv25_subsampled)}")

    converter = VTKToUSDConverter(settings)

    tpv25_files_sub = [file_path for _, file_path in tpv25_subsampled]
    tpv25_times_sub = [float(time_step) for time_step, _ in tpv25_subsampled]

    output_usd_sub = output_dir / f"tpv25_subsample_{subsample_rate}x.usd"

    print(f"\nConverting to: {output_usd_sub}")

    start_time = time_module.time()

    # Read MeshData
    mesh_data_sequence = [
        read_vtk_file(f, extract_surface=True) for f in tpv25_files_sub
    ]

    # Validate topology consistency across time series
    validation_report = validate_time_series_topology(
        mesh_data_sequence, filenames=tpv25_files_sub
    )
    if not validation_report["is_consistent"]:
        print(
            f"Warning: Found {len(validation_report['warnings'])} topology/primvar issues"
        )
        if validation_report["topology_changes"]:
            print(
                f"  Topology changes in {len(validation_report['topology_changes'])} frames"
            )

    # Convert to USD (preserves all primvars from VTK)
    stage_sub = converter.convert_mesh_data_sequence(
        mesh_data_sequence=mesh_data_sequence,
        output_usd=output_usd_sub,
        mesh_name="TPV25Valve",
        time_codes=tpv25_times_sub,
        material=tpv25_material,
    )

    # Post-process: apply colormap visualization using USDTools
    if ENABLE_AUTO_COLORIZATION:
        usd_tools = USDTools()
        mesh_path = "/World/Meshes/TPV25Valve"

        # Inspect and select primvar for coloring
        primvars = usd_tools.list_mesh_primvars(str(output_usd_sub), mesh_path)
        color_primvar = usd_tools.pick_color_primvar(
            primvars, keywords=("strain", "stress")
        )

        if color_primvar:
            print(f"\nApplying colormap to '{color_primvar}' using {DEFAULT_COLORMAP}")
            usd_tools.apply_colormap_from_primvar(
                str(output_usd_sub),
                mesh_path,
                color_primvar,
                cmap=DEFAULT_COLORMAP,
                bind_vertex_color_material=True,
            )
        else:
            print("\nNo strain/stress primvar found for coloring")

    elapsed = time_module.time() - start_time

    print(f"\n✓ Conversion completed in {elapsed:.1f} seconds")
    print(f"  Output: {output_usd_sub}")
    print(f"  Size: {output_usd_sub.stat().st_size / (1024 * 1024):.2f} MB")

INFO: Reading legacy VTK file: c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\TPV25\TPV25_output.t000.vtk


Converting Subsampled TPV25 (every 5th frame)
Frames: 265 → 53

Converting to: c:\src\Projects\PhysioMotion\physiomotion4d\experiments\convert_vtk_to_usd_lib\output\valve4d\tpv25_subsample_5x.usd


INFO: Loaded mesh: 38301 points, 63332 faces, 4 data arrays
INFO: Reading legacy VTK file: c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\TPV25\TPV25_output.t005.vtk
INFO: Loaded mesh: 38301 points, 63332 faces, 4 data arrays
INFO: Reading legacy VTK file: c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\TPV25\TPV25_output.t010.vtk
INFO: Loaded mesh: 38301 points, 63332 faces, 4 data arrays
INFO: Reading legacy VTK file: c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\TPV25\TPV25_output.t015.vtk
INFO: Loaded mesh: 38301 points, 63332 faces, 4 data arrays
INFO: Reading legacy VTK file: c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\TPV25\TPV25_output.t020.vtk
INFO: Loaded mesh: 38301 points, 63332 faces, 4 data arrays
INFO: Reading legacy VTK file: c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\TPV25\TPV25_output.t025.vtk
INFO: Loaded mesh: 38301 points, 63332 faces, 4 data arrays
INFO: Reading legacy VTK file: c:\s


Applying colormap to 'vtk_point_stress_c0' using plasma


2026-01-29 21:23:31 INFO USDTools Wrote displayColor primvar with 53 time samples
2026-01-29 21:23:31 INFO USDTools Created vertex color material: /World/Looks/VertexColorMaterial
2026-01-29 21:23:31 INFO USDTools Saved USD file: c:\src\Projects\PhysioMotion\physiomotion4d\experiments\convert_vtk_to_usd_lib\output\valve4d\tpv25_subsample_5x.usd



✓ Conversion completed in 62.4 seconds
  Output: c:\src\Projects\PhysioMotion\physiomotion4d\experiments\convert_vtk_to_usd_lib\output\valve4d\tpv25_subsample_5x.usd
  Size: 176.81 MB


## 7. Create Combined Scene

Create a single USD file with both valves side-by-side for comparison.

In [15]:
# Create combined scene with both valves using USDTools
if alterra_series and tpv25_series:
    print("=" * 60)
    print("Creating Combined Scene")
    print("=" * 60)

    # Use the subsampled USD files created earlier
    subsample_rate = 5
    alterra_usd = output_dir / f"alterra_subsample_{subsample_rate}x.usd"
    tpv25_usd = output_dir / f"tpv25_subsample_{subsample_rate}x.usd"

    # Check if the files exist
    if alterra_usd.exists() and tpv25_usd.exists():
        combined_usd = output_dir / "valves_combined.usd"

        print("Input files:")
        print(f"  - {alterra_usd.name}")
        print(f"  - {tpv25_usd.name}")
        print(f"Output: {combined_usd.name}")

        # Use USDTools to arrange the valves side-by-side
        from physiomotion4d.usd_tools import USDTools

        usd_tools = USDTools()

        usd_tools.save_usd_file_arrangement(
            str(combined_usd), [str(alterra_usd), str(tpv25_usd)]
        )

        print(f"\n✓ Combined scene created: {combined_usd.name}")
        print("  - Both valves arranged in a spatial grid")
        print("  - Ready to view in Omniverse or USDView")
    else:
        print("\n⚠ Subsampled USD files not found.")
        print("Run the conversion cells above first to create:")
        print(f"  - {alterra_usd.name}")
        print(f"  - {tpv25_usd.name}")

2026-01-29 21:23:31 INFO USDTools Grid layout: 1 rows x 2 cols
2026-01-29 21:23:31 INFO USDTools Copying c:\src\Projects\PhysioMotion\physiomotion4d\experiments\convert_vtk_to_usd_lib\output\valve4d\alterra_subsample_5x.usd:/World/Looks
2026-01-29 21:23:31 WARNING USDTools No valid bounding box found for prim: /World/Looks/VertexColorMaterial
2026-01-29 21:23:31 INFO USDTools Copying c:\src\Projects\PhysioMotion\physiomotion4d\experiments\convert_vtk_to_usd_lib\output\valve4d\alterra_subsample_5x.usd:/World/Meshes
2026-01-29 21:23:31 WARNING USDTools No valid bounding box found for prim: /World/Meshes/AlterraValve
2026-01-29 21:23:31 INFO USDTools Copying c:\src\Projects\PhysioMotion\physiomotion4d\experiments\convert_vtk_to_usd_lib\output\valve4d\tpv25_subsample_5x.usd:/World/Looks
2026-01-29 21:23:31 WARNING USDTools No valid bounding box found for prim: /World/Looks/VertexColorMaterial
2026-01-29 21:23:31 INFO USDTools Copying c:\src\Projects\PhysioMotion\physiomotion4d\experiments\

Creating Combined Scene
Input files:
  - alterra_subsample_5x.usd
  - tpv25_subsample_5x.usd
Output: valves_combined.usd

✓ Combined scene created: valves_combined.usd
  - Both valves arranged in a spatial grid
  - Ready to view in Omniverse or USDView


## 8. Summary and File Inspection

In [16]:
import os

print("=" * 60)
print("Conversion Summary")
print("=" * 60)

# List all generated USD files
usd_files = list(output_dir.glob("*.usd"))
usd_files.extend(output_dir.glob("*.usda"))
usd_files.extend(output_dir.glob("*.usdc"))

total_size = 0

for usd_file in sorted(usd_files):
    size_mb = os.path.getsize(usd_file) / (1024 * 1024)
    total_size += size_mb

    print(f"\n{usd_file.name}:")
    print(f"  Size: {size_mb:.2f} MB")

    # Open and inspect
    stage = Usd.Stage.Open(str(usd_file))
    if stage:
        if stage.HasAuthoredTimeCodeRange():
            duration = (
                stage.GetEndTimeCode() - stage.GetStartTimeCode()
            ) / stage.GetTimeCodesPerSecond()
            print(
                f"  Time range: {stage.GetStartTimeCode():.0f} - {stage.GetEndTimeCode():.0f}"
            )
            print(
                f"  Duration: {duration:.2f} seconds @ {stage.GetTimeCodesPerSecond():.0f} FPS"
            )
            print(
                f"  Frames: {int(stage.GetEndTimeCode() - stage.GetStartTimeCode() + 1)}"
            )

        # Count meshes
        mesh_count = 0
        for prim in stage.Traverse():
            if prim.IsA(UsdGeom.Mesh):
                mesh_count += 1
        print(f"  Meshes: {mesh_count}")

print(f"\n{'=' * 60}")
print(f"Total size: {total_size:.2f} MB")
print(f"Total files: {len(usd_files)}")
print(f"Output directory: {output_dir}")
print(f"{'=' * 60}")

Conversion Summary

alterra_subsample_5x.usd:
  Size: 198.03 MB
  Time range: 0 - 230
  Duration: 3.83 seconds @ 60 FPS
  Frames: 231
  Meshes: 1

alterra_subsample_5x_colored.usd:
  Size: 224.11 MB
  Time range: 0 - 230
  Duration: 3.83 seconds @ 60 FPS
  Frames: 231
  Meshes: 1

tpv25_subsample_5x.usd:
  Size: 176.81 MB
  Time range: 0 - 260
  Duration: 4.33 seconds @ 60 FPS
  Frames: 261
  Meshes: 1

valves_combined.usd:
  Size: 374.84 MB
  Time range: 0 - 230
  Duration: 3.83 seconds @ 60 FPS
  Frames: 231
  Meshes: 2

Total size: 973.80 MB
Total files: 4
Output directory: c:\src\Projects\PhysioMotion\physiomotion4d\experiments\convert_vtk_to_usd_lib\output\valve4d


## 9. Detailed USD Inspection

Examine the converted USD files to verify data preservation.

In [17]:
# Inspect one of the converted files in detail
inspect_file = output_dir / "alterra_subsample_5x.usd"

if inspect_file.exists():
    print("=" * 60)
    print(f"Detailed Inspection: {inspect_file.name}")
    print("=" * 60)

    stage = Usd.Stage.Open(str(inspect_file))

    # Find mesh prim
    mesh_prim = None
    for prim in stage.Traverse():
        if prim.IsA(UsdGeom.Mesh):
            mesh_prim = prim
            break

    if mesh_prim:
        mesh = UsdGeom.Mesh(mesh_prim)

        print(f"\nMesh: {mesh_prim.GetPath()}")

        # Geometry at first frame
        first_time = stage.GetStartTimeCode()
        points = mesh.GetPointsAttr().Get(first_time)
        faces = mesh.GetFaceVertexCountsAttr().Get()

        print(f"\nGeometry (at t={first_time:.0f}):")
        print(f"  Points: {len(points):,}")
        print(f"  Faces: {len(faces):,}")

        # Check time-varying attributes
        print("\nTime-Varying Attributes:")
        points_attr = mesh.GetPointsAttr()
        if points_attr.GetNumTimeSamples() > 0:
            print(f"  Points: {points_attr.GetNumTimeSamples()} time samples")

        # List primvars
        primvars_api = UsdGeom.PrimvarsAPI(mesh)
        primvars = primvars_api.GetPrimvars()

        print(f"\nPrimvars ({len(primvars)}):")
        for primvar in primvars:
            name = primvar.GetPrimvarName()
            interpolation = primvar.GetInterpolation()
            type_name = primvar.GetTypeName()
            value = primvar.Get(first_time)
            size = len(value) if value else 0

            print(f"  - {name}:")
            print(f"      Type: {type_name}")
            print(f"      Interpolation: {interpolation}")
            print(f"      Elements: {size:,}")

            # Check if time-varying
            if primvar.GetAttr().GetNumTimeSamples() > 0:
                print(f"      Time samples: {primvar.GetAttr().GetNumTimeSamples()}")

        # Material binding
        from pxr import UsdShade

        binding_api = UsdShade.MaterialBindingAPI(mesh)
        material_binding = binding_api.GetDirectBinding()
        if material_binding:
            print(f"\nMaterial: {material_binding.GetMaterialPath()}")
else:
    print(f"File not found: {inspect_file}")

Detailed Inspection: alterra_subsample_5x.usd

Mesh: /World/Meshes/AlterraValve

Geometry (at t=0):
  Points: 48,482
  Faces: 96,888

Time-Varying Attributes:
  Points: 47 time samples

Primvars (8):
  - displayColor:
      Type: color3f[]
      Interpolation: vertex
      Elements: 48,482
      Time samples: 47
  - displayOpacity:
      Type: float[]
      Interpolation: constant
      Elements: 0
  - vtk_point_displacement:
      Type: float3[]
      Interpolation: vertex
      Elements: 48,482
      Time samples: 47
  - vtk_point_relative_volume:
      Type: float[]
      Interpolation: vertex
      Elements: 48,482
      Time samples: 47
  - vtk_point_shell_thickness:
      Type: float[]
      Interpolation: vertex
      Elements: 48,482
      Time samples: 47
  - vtk_point_stress_c0:
      Type: float3[]
      Interpolation: vertex
      Elements: 48,482
      Time samples: 47
  - vtk_point_stress_c1:
      Type: float3[]
      Interpolation: vertex
      Elements: 48,482
      Ti

## 9.5. Post-Process USD with USDTools

Demonstrate using the new `USDTools` methods to inspect primvars and apply colormap visualization to existing USD files.

In [18]:
# Example: Post-process an existing USD file to add colormap visualization
from physiomotion4d.usd_tools import USDTools

usd_tools = USDTools()

# Pick a USD file to post-process
postprocess_file = output_dir / "alterra_subsample_5x.usd"

if postprocess_file.exists():
    print("=" * 60)
    print(f"Post-Processing: {postprocess_file.name}")
    print("=" * 60)

    # 1. List available primvars on the mesh
    mesh_path = "/World/Meshes/AlterraValve"
    primvars = usd_tools.list_mesh_primvars(str(postprocess_file), mesh_path)

    print(f"\nAvailable primvars on {mesh_path}:")
    for pv in primvars:
        time_info = (
            f", {pv['num_time_samples']} time samples"
            if pv["num_time_samples"] > 0
            else ""
        )
        range_info = (
            f", range={pv['range'][0]:.3g}..{pv['range'][1]:.3g}" if pv["range"] else ""
        )
        print(
            f"  - {pv['name']}: {pv['interpolation']}, {pv['elements']} elements{time_info}{range_info}"
        )

    # 2. Pick best primvar for coloring (prefer strain/stress)
    color_primvar = usd_tools.pick_color_primvar(primvars)
    print(f"\nAuto-selected for coloring: {color_primvar}")

    # 3. Apply colormap to create displayColor visualization
    # Note: This modifies the USD file in-place
    if color_primvar:
        print(f"\nApplying 'plasma' colormap to '{color_primvar}'...")

        # Create a copy for demonstration (optional)
        demo_file = output_dir / f"{postprocess_file.stem}_colored.usd"
        import shutil

        shutil.copy(postprocess_file, demo_file)

        usd_tools.apply_colormap_from_primvar(
            str(demo_file),
            mesh_path,
            color_primvar,
            cmap="plasma",
            write_default_at_t0=True,
            bind_vertex_color_material=True,
        )

        print(f"\n✓ Created colored visualization: {demo_file.name}")
        print(f"  - displayColor primvar added with colormap from {color_primvar}")
        print("  - Vertex color material bound for immediate visualization")
        print("  - Ready to open in Omniverse with default coloring")
    else:
        print("\n⚠️  No suitable primvar found for coloring")
else:
    print(f"File not found: {postprocess_file}")
    print("Run the conversion cells first to generate USD files.")

2026-01-29 21:23:31 INFO USDTools Processing 47 time samples for primvar 'vtk_point_stress_c0'


Post-Processing: alterra_subsample_5x.usd

Available primvars on /World/Meshes/AlterraValve:
  - displayColor: vertex, 48482 elements, 47 time samples, range=0.0298..0.528
  - displayOpacity: constant, 0 elements
  - vtk_point_displacement: vertex, 0 elements, 47 time samples
  - vtk_point_relative_volume: vertex, 0 elements, 47 time samples
  - vtk_point_shell_thickness: vertex, 0 elements, 47 time samples
  - vtk_point_stress_c0: vertex, 0 elements, 47 time samples
  - vtk_point_stress_c1: vertex, 0 elements, 47 time samples
  - vtk_point_stress_c2: vertex, 0 elements, 47 time samples

Auto-selected for coloring: vtk_point_stress_c0

Applying 'plasma' colormap to 'vtk_point_stress_c0'...


2026-01-29 21:23:32 INFO USDTools Value range: 0 to 8.5216e+07
2026-01-29 21:23:35 INFO USDTools Wrote displayColor primvar with 47 time samples
2026-01-29 21:23:35 INFO USDTools Saved USD file: c:\src\Projects\PhysioMotion\physiomotion4d\experiments\convert_vtk_to_usd_lib\output\valve4d\alterra_subsample_5x_colored.usd



✓ Created colored visualization: alterra_subsample_5x_colored.usd
  - displayColor primvar added with colormap from vtk_point_stress_c0
  - Vertex color material bound for immediate visualization
  - Ready to open in Omniverse with default coloring


## 10. Performance Analysis

In [19]:
# Analyze conversion performance
print("=" * 60)
print("Performance Analysis")
print("=" * 60)

# Read a few frames to estimate per-frame metrics
if alterra_series:
    sample_files = [
        alterra_series[0][1],
        alterra_series[len(alterra_series) // 2][1],
        alterra_series[-1][1],
    ]

    total_points = 0
    total_faces = 0
    total_arrays = 0

    for sample_file in sample_files:
        mesh_data = read_vtk_file(sample_file, extract_surface=True)
        total_points += len(mesh_data.points)
        total_faces += len(mesh_data.face_vertex_counts)
        total_arrays += len(mesh_data.generic_arrays)

    avg_points = total_points / len(sample_files)
    avg_faces = total_faces / len(sample_files)
    avg_arrays = total_arrays / len(sample_files)

    print("\nAlterra Dataset:")
    print(f"  Average points per frame: {avg_points:,.0f}")
    print(f"  Average faces per frame: {avg_faces:,.0f}")
    print(f"  Average data arrays per frame: {avg_arrays:.0f}")
    print(f"  Total frames: {len(alterra_series)}")
    print(f"  Estimated total points: {avg_points * len(alterra_series):,.0f}")
    print(f"  Estimated total faces: {avg_faces * len(alterra_series):,.0f}")

print(f"\n{'=' * 60}")
print("\n✓ All conversions completed!")
print("\nView the results:")
print("  - USDView: usdview <filename>.usd")
print("  - Omniverse: Open in Create/View/Composer")
print(f"\nOutput files: {output_dir}")
print("=" * 60)

INFO: Reading legacy VTK file: c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\Alterra\Alterra_output.t000.vtk


Performance Analysis


INFO: Loaded mesh: 48482 points, 83634 faces, 4 data arrays
INFO: Reading legacy VTK file: c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\Alterra\Alterra_output.t116.vtk
INFO: Loaded mesh: 48482 points, 83634 faces, 4 data arrays
INFO: Reading legacy VTK file: c:\src\Projects\PhysioMotion\physiomotion4d\data\CHOP-Valve4D\Alterra\Alterra_output.t231.vtk
INFO: Loaded mesh: 48482 points, 83634 faces, 4 data arrays



Alterra Dataset:
  Average points per frame: 48,482
  Average faces per frame: 83,634
  Average data arrays per frame: 4
  Total frames: 232
  Estimated total points: 11,247,824
  Estimated total faces: 19,403,088


✓ All conversions completed!

View the results:
  - USDView: usdview <filename>.usd
  - Omniverse: Open in Create/View/Composer

Output files: c:\src\Projects\PhysioMotion\physiomotion4d\experiments\convert_vtk_to_usd_lib\output\valve4d


## Conclusion

This notebook demonstrated converting large-scale time-varying cardiac valve simulation data to USD:

### Key Accomplishments

1. **Discovered and organized** 200+ frame time-series datasets
2. **Converted full-resolution** datasets to animated USD
3. **Created subsampled versions** for faster preview
4. **Preserved all simulation data** as USD primvars
5. **Applied custom materials** for visualization
6. **Handled coordinate systems** (RAS → Y-up)

### File Outputs

- `alterra_full.usd` - Complete 232-frame animation
- `alterra_subsample_5x.usd` - Subsampled for preview
- `tpv25_full.usd` - Complete 265-frame animation
- `tpv25_subsample_5x.usd` - Subsampled for preview

### Performance Notes

- Full conversions may take several minutes due to large frame counts
- Subsampling provides faster iteration during development
- All VTK point and cell data arrays are preserved as primvars
- Time-sampled attributes enable efficient animation

### Next Steps

1. **View animations** in USDView or Omniverse
2. **Analyze primvars** to visualize simulation data
3. **Create custom materials** based on data arrays
4. **Compose scenes** with multiple valves for comparison
5. **Add cameras and lighting** for publication-quality renders